In [0]:
import xml.etree.ElementTree as ET

In [0]:
%sh
pip install spark-xml

ERROR: Could not find a version that satisfies the requirement spark-xml (from versions: none)
ERROR: No matching distribution found for spark-xml
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-425920c6-12d2-4716-bab5-ea8af7b78da3/bin/python -m pip install --upgrade pip' command.


# Connection to blob

In [0]:
storageAccount = "datalakefabdeprojetos"
containerName = "ingrid-sollim"
mountpoint = "/mnt/ingrid-sollim/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope="Kv-fab-de-proj-dev",key="kv-ingrid-sollim")
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"
try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails


Mount point '/mnt/ingrid-sollim/' is already mounted.


In [0]:
path=mountpoint+"bikes"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
dbfs:/mnt/ingrid-sollim/bikes/customer.csv,customer.csv,1813963,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/person.csv,person.csv,13646947,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/product.csv,product.csv,104823,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderdetail.csv,salesorderdetail.csv,13801182,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderheader.csv,salesorderheader.csv,8267704,1694474862000
dbfs:/mnt/ingrid-sollim/bikes/spoffer.csv,spoffer.csv,36680,1694474860000


In [0]:
df = spark.read.option("header",True).options(delimiter=";").csv(path+"/spoffer.csv")
display(df.limit(10))

SpecialOfferID,ProductID,rowguid,ModifiedDate
1,680,BB30B868-D86C-4557-8DB2-4B2D0A83A0FB,2011-04-01 00:00:00.000
1,706,B3C9A4B1-2AE6-4CBA-B552-1F206C9F4C1F,2011-04-01 00:00:00.000
1,707,27B711FE-0B77-4EA4-AD1A-7C239956BEF4,2011-04-01 00:00:00.000
1,708,46CBB78B-246E-4D69-9BD6-E521277C1078,2011-04-01 00:00:00.000
1,709,CF102AA0-055F-4D2B-8B98-04B161758EA8,2011-04-01 00:00:00.000
1,710,63718DA1-464B-4325-9514-CDEE46CB124F,2011-04-01 00:00:00.000
1,711,457EB971-D1C9-48CA-B947-AE7E1B114377,2011-04-01 00:00:00.000
1,712,5B948448-BAE5-4F2A-A1F3-8203E892FD24,2011-04-01 00:00:00.000
1,713,07768F40-6E46-430F-AC1A-FF6A3629729C,2011-04-01 00:00:00.000
1,714,85004BCE-C74A-4D4E-8D17-3157991A1400,2011-04-01 00:00:00.000


In [0]:
df.printSchema()

root
 |-- SpecialOfferID: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [0]:
display(df.select("ModifiedDate").distinct())

ModifiedDate
2011-04-01 00:00:00.000
2014-03-01 00:00:00.000
2012-04-30 00:00:00.000
2013-04-30 00:00:00.000
2012-03-14 00:00:00.000
2011-05-01 00:00:00.000
2013-06-14 00:00:00.000
2013-04-14 00:00:00.000


In [0]:
from pyspark.sql.functions import expr, col, regexp_replace,regexp_extract,when, to_date,to_timestamp

#Change Data types

In [0]:
df_data = df.withColumn("SpecialOfferID",col("SpecialOfferID").cast("int"))\
.withColumn("ProductID",col("ProductID").cast("int"))\
.withColumn("ModifiedDate",to_date(to_timestamp(col("ModifiedDate"), "yyyy-MM-dd HH:mm:ss.SSS"),"yyyy-MM-dd"))

display(df_data.limit(10))

SpecialOfferID,ProductID,rowguid,ModifiedDate
1,680,BB30B868-D86C-4557-8DB2-4B2D0A83A0FB,2011-04-01
1,706,B3C9A4B1-2AE6-4CBA-B552-1F206C9F4C1F,2011-04-01
1,707,27B711FE-0B77-4EA4-AD1A-7C239956BEF4,2011-04-01
1,708,46CBB78B-246E-4D69-9BD6-E521277C1078,2011-04-01
1,709,CF102AA0-055F-4D2B-8B98-04B161758EA8,2011-04-01
1,710,63718DA1-464B-4325-9514-CDEE46CB124F,2011-04-01
1,711,457EB971-D1C9-48CA-B947-AE7E1B114377,2011-04-01
1,712,5B948448-BAE5-4F2A-A1F3-8203E892FD24,2011-04-01
1,713,07768F40-6E46-430F-AC1A-FF6A3629729C,2011-04-01
1,714,85004BCE-C74A-4D4E-8D17-3157991A1400,2011-04-01


In [0]:
column_mapping = {
'SpecialOfferID':'special_offer_id',
 'ProductID':'product_id',
 'rowguid':'rowguid',
 'ModifiedDate':'modified_date'
}

# Create a list of column expressions with aliases
select_expr = [col(old_col).alias(new_col) for old_col, new_col in column_mapping.items()]

# Apply the column renaming using select and alias
special_offer = df_data.select(*select_expr)
display(special_offer.limit(10))

special_offer_id,product_id,rowguid,modified_date
1,680,BB30B868-D86C-4557-8DB2-4B2D0A83A0FB,2011-04-01
1,706,B3C9A4B1-2AE6-4CBA-B552-1F206C9F4C1F,2011-04-01
1,707,27B711FE-0B77-4EA4-AD1A-7C239956BEF4,2011-04-01
1,708,46CBB78B-246E-4D69-9BD6-E521277C1078,2011-04-01
1,709,CF102AA0-055F-4D2B-8B98-04B161758EA8,2011-04-01
1,710,63718DA1-464B-4325-9514-CDEE46CB124F,2011-04-01
1,711,457EB971-D1C9-48CA-B947-AE7E1B114377,2011-04-01
1,712,5B948448-BAE5-4F2A-A1F3-8203E892FD24,2011-04-01
1,713,07768F40-6E46-430F-AC1A-FF6A3629729C,2011-04-01
1,714,85004BCE-C74A-4D4E-8D17-3157991A1400,2011-04-01


In [0]:
special_offer.printSchema()

root
 |-- special_offer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- modified_date: date (nullable = true)



#Ingest the cleaned data to the database

### Connection to the database

In [0]:
# Get the secrets from the jdbc secret scope.
username = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dblogin")
password = dbutils.secrets.get(scope="Kv-fab-de-proj-dev", key="kv-soll-dbpass")

jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "db-fab-projetos"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : username,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.special_offer'

### Insert into the database


In [0]:
special_offer.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()

In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[17]: True